<a href="https://colab.research.google.com/github/CrockPS/CSGO-data/blob/main/CsgoData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importações

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import numpy as np

#Realizando toda a configuração inicial da base incluindo divisão para treino e teste

In [4]:
csgo_data = pd.read_csv('/content/csgo_round_snapshots.csv')

In [5]:
csgo_data.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,t_helmets,ct_defuse_kits,ct_players_alive,t_players_alive,ct_weapon_ak47,t_weapon_ak47,ct_weapon_aug,t_weapon_aug,ct_weapon_awp,t_weapon_awp,ct_weapon_bizon,t_weapon_bizon,ct_weapon_cz75auto,t_weapon_cz75auto,ct_weapon_elite,t_weapon_elite,ct_weapon_famas,t_weapon_famas,ct_weapon_g3sg1,t_weapon_g3sg1,ct_weapon_galilar,t_weapon_galilar,ct_weapon_glock,t_weapon_glock,ct_weapon_m249,t_weapon_m249,ct_weapon_m4a1s,t_weapon_m4a1s,...,t_weapon_p90,ct_weapon_r8revolver,t_weapon_r8revolver,ct_weapon_sawedoff,t_weapon_sawedoff,ct_weapon_scar20,t_weapon_scar20,ct_weapon_sg553,t_weapon_sg553,ct_weapon_ssg08,t_weapon_ssg08,ct_weapon_ump45,t_weapon_ump45,ct_weapon_xm1014,t_weapon_xm1014,ct_weapon_deagle,t_weapon_deagle,ct_weapon_fiveseven,t_weapon_fiveseven,ct_weapon_usps,t_weapon_usps,ct_weapon_p250,t_weapon_p250,ct_weapon_p2000,t_weapon_p2000,ct_weapon_tec9,t_weapon_tec9,ct_grenade_hegrenade,t_grenade_hegrenade,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,4000.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,650.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,500.0,0.0,0.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,10750.0,0.0,0.0,1.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT


In [6]:
csgo_data['round_winner'].value_counts()

T     62406
CT    60004
Name: round_winner, dtype: int64

In [7]:
csgo_data['round_winner'] = csgo_data['round_winner'].replace(['T'], 0)
csgo_data['round_winner'] = csgo_data['round_winner'].replace(['CT'], 1)

In [8]:
x_csgo_data = csgo_data.iloc[:, 0:96].values
x_csgo_data

array([[175.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [156.03, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [96.03, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [114.93, 11.0, 15.0, ..., 5.0, 0.0, 0.0],
       [94.93, 11.0, 15.0, ..., 4.0, 0.0, 0.0],
       [74.93, 11.0, 15.0, ..., 3.0, 0.0, 0.0]], dtype=object)

In [9]:
y_csgo_data = csgo_data.iloc[:, 96].values
y_csgo_data

array([1, 1, 1, ..., 0, 0, 0])

In [10]:
label_encoder_map = LabelEncoder()
label_encoder_bomb_planted = LabelEncoder()

x_csgo_data[:,3] = label_encoder_map.fit_transform(x_csgo_data[:,3])
x_csgo_data[:,4] = label_encoder_bomb_planted.fit_transform(x_csgo_data[:,4])

In [11]:
x_csgo_data

array([[175.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [156.03, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [96.03, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [114.93, 11.0, 15.0, ..., 5.0, 0.0, 0.0],
       [94.93, 11.0, 15.0, ..., 4.0, 0.0, 0.0],
       [74.93, 11.0, 15.0, ..., 3.0, 0.0, 0.0]], dtype=object)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x_csgo_data, y_csgo_data, test_size=0.33, random_state=42, shuffle=True)


####Salvando nossa base já configurada como um arquivo .pkl para utilizações futuras

In [13]:
import pickle
with open('csgo_data.pkl', 'wb') as f:
  pickle.dump([x_csgo_data, y_csgo_data], f)

#Algoritmo Naive Bayes

In [14]:
naive_csgo_data = GaussianNB()
naive_csgo_data.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [15]:
previsaoNB = naive_csgo_data.predict(x_test)
previsaoNB

array([0, 1, 1, ..., 1, 0, 1])

###Analise geral Naive Bayes

In [16]:
print(f'Acurácia: {metrics.accuracy_score(y_test, previsaoNB)}')
print(f'Recall: {metrics.recall_score(y_test, previsaoNB)}')
print(f'Precisão: {metrics.precision_score(y_test, previsaoNB)}')
print(f'F1-Score: {metrics.f1_score(y_test, previsaoNB)}')
print('')
print(f'Resumo: {metrics.classification_report(y_test, previsaoNB)}')

Acurácia: 0.7318298841469453
Recall: 0.7339569503118085
Precisão: 0.72473556140438
F1-Score: 0.729317108518028

Resumo:               precision    recall  f1-score   support

           0       0.74      0.73      0.73     20512
           1       0.72      0.73      0.73     19884

    accuracy                           0.73     40396
   macro avg       0.73      0.73      0.73     40396
weighted avg       0.73      0.73      0.73     40396



#Algoritmo Árvores de Decisão

In [17]:
arvore_csgo_data = DecisionTreeClassifier('entropy')
arvore_csgo_data.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [18]:
#arvore_csgo_data.classes_
#figura, eixos = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
#tree.plot_tree(arvore_csgo_data, filled=True);

In [19]:
previsaoTree = arvore_csgo_data.predict(x_test)
previsaoTree

array([1, 0, 1, ..., 1, 1, 1])

###Analise geral Árvores de Decisão

In [20]:
print(f'Acurácia: {metrics.accuracy_score(y_test, previsaoTree)}')
print(f'Recall: {metrics.recall_score(y_test, previsaoTree)}')
print(f'Precisão: {metrics.precision_score(y_test, previsaoTree)}')
print(f'F1-Score: {metrics.f1_score(y_test, previsaoTree)}')
print('')
print(f'Resumo: {metrics.classification_report(y_test, previsaoTree)}')

Acurácia: 0.8142885434201406
Recall: 0.8032086099376383
Precisão: 0.8165132924335379
F1-Score: 0.8098063076767061

Resumo:               precision    recall  f1-score   support

           0       0.81      0.83      0.82     20512
           1       0.82      0.80      0.81     19884

    accuracy                           0.81     40396
   macro avg       0.81      0.81      0.81     40396
weighted avg       0.81      0.81      0.81     40396



#Algoritmo KNN

In [21]:
knn_csgo_data = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p = 2)
knn_csgo_data.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [22]:
previsaoKNN = knn_csgo_data.predict(x_test)
previsaoKNN

array([0, 1, 1, ..., 1, 1, 1])

###Analise geral KNN

In [23]:
print(f'Acurácia: {metrics.accuracy_score(y_test, previsaoKNN)}')
print(f'Recall: {metrics.recall_score(y_test, previsaoKNN)}')
print(f'Precisão: {metrics.precision_score(y_test, previsaoKNN)}')
print(f'F1-Score: {metrics.f1_score(y_test, previsaoKNN)}')
print('')
print(f'Resumo: {metrics.classification_report(y_test, previsaoKNN)}')

Acurácia: 0.749925735221309
Recall: 0.745876081271374
Precisão: 0.7460261569416499
F1-Score: 0.7459511115581934

Resumo:               precision    recall  f1-score   support

           0       0.75      0.75      0.75     20512
           1       0.75      0.75      0.75     19884

    accuracy                           0.75     40396
   macro avg       0.75      0.75      0.75     40396
weighted avg       0.75      0.75      0.75     40396



#Algoritmo SVM

In [26]:
svm_csgo_data = SVC()
svm_csgo_data.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [27]:
previsaoSVM = svm_csgo_data.predict(x_test)
previsaoSVM

array([1, 0, 1, ..., 1, 0, 1])

###Analise geral SVM

In [29]:
print(f'Acurácia: {metrics.accuracy_score(y_test, previsaoSVM)}')
print(f'Recall: {metrics.recall_score(y_test, previsaoSVM)}')
print(f'Precisão: {metrics.precision_score(y_test, previsaoSVM)}')
print(f'F1-Score: {metrics.f1_score(y_test, previsaoSVM)}')
print('')
print(f'Resumo: {metrics.classification_report(y_test, previsaoSVM)}')

Acurácia: 0.7220516882859689
Recall: 0.7025749346208007
Precisão: 0.7244347645716657
F1-Score: 0.7133374183006537

Resumo:               precision    recall  f1-score   support

           0       0.72      0.74      0.73     20512
           1       0.72      0.70      0.71     19884

    accuracy                           0.72     40396
   macro avg       0.72      0.72      0.72     40396
weighted avg       0.72      0.72      0.72     40396



# Analise Geral
Com base nos dados sobre o jogo CS GO que analisamos é possível calcular com uma taxa boa de sucesso qual será o possível time vencedor, e que o time "Terrorista", representado por 0, será o ganhador na maioria das vezes devido a diversos fatores com uma pequena diferença percentual, uma analise em tempo real seria possível, porém iria requerer a entrada de muitos dados.

Após analisar a implementação dos 4 algoritmos é possível perceber que o tem os melhores resultados é o de 'Árvores de Decisão'

*Segue abaixo os resultados dos respectivos algoritmos:*

In [37]:
print(f'> Acurácia:')
print(f'---------- Naive Bayes: {metrics.accuracy_score(y_test, previsaoNB)}')
print(f'---------- Árvores de Decisão: {metrics.accuracy_score(y_test, previsaoTree)}')
print(f'---------- k-nearest neighbors (KNN): {metrics.accuracy_score(y_test, previsaoKNN)}')
print(f'---------- Support vector machine (SVM): {metrics.accuracy_score(y_test, previsaoSVM)}')
print('')
print(f'> Recall:')
print(f'---------- Naive Bayes: {metrics.recall_score(y_test, previsaoNB)}')
print(f'---------- Árvores de Decisão: {metrics.recall_score(y_test, previsaoTree)}')
print(f'---------- k-nearest neighbors (KNN): {metrics.recall_score(y_test, previsaoKNN)}')
print(f'---------- Support vector machine (SVM): {metrics.recall_score(y_test, previsaoSVM)}')
print('')
print(f'> Precisão:')
print(f'---------- Naive Bayes: {metrics.precision_score(y_test, previsaoNB)}')
print(f'---------- Árvores de Decisão: {metrics.precision_score(y_test, previsaoTree)}')
print(f'---------- k-nearest neighbors (KNN): {metrics.precision_score(y_test, previsaoKNN)}')
print(f'---------- Support vector machine (SVM): {metrics.precision_score(y_test, previsaoSVM)}')
print('')
print(f'> F1-Score:')
print(f'---------- Naive Bayes: {metrics.f1_score(y_test, previsaoNB)}')
print(f'---------- Árvores de Decisão: {metrics.f1_score(y_test, previsaoTree)}')
print(f'---------- k-nearest neighbors (KNN): {metrics.f1_score(y_test, previsaoKNN)}')
print(f'---------- Support vector machine (SVM): {metrics.f1_score(y_test, previsaoSVM)}')
print('')
print(f'Resumo:')
print('')
print(f'---Naive Bayes:')
print(f'{metrics.classification_report(y_test, previsaoNB)}')
print('')
print(f'---Árvores de Decisão:')
print(f'{metrics.classification_report(y_test, previsaoTree)}')
print('')
print(f'---k-nearest neighbors (KNN):')
print(f'{metrics.classification_report(y_test, previsaoKNN)}')
print('')
print(f'---Support vector machine (SVM):')
print(f'{metrics.classification_report(y_test, previsaoSVM)}')

> Acurácia:
---------- Naive Bayes: 0.7318298841469453
---------- Árvores de Decisão: 0.8142885434201406
---------- k-nearest neighbors (KNN): 0.749925735221309
---------- Support vector machine (SVM): 0.7220516882859689

> Recall:
---------- Naive Bayes: 0.7339569503118085
---------- Árvores de Decisão: 0.8032086099376383
---------- k-nearest neighbors (KNN): 0.745876081271374
---------- Support vector machine (SVM): 0.7025749346208007

> Precisão:
---------- Naive Bayes: 0.72473556140438
---------- Árvores de Decisão: 0.8165132924335379
---------- k-nearest neighbors (KNN): 0.7460261569416499
---------- Support vector machine (SVM): 0.7244347645716657

> F1-Score:
---------- Naive Bayes: 0.729317108518028
---------- Árvores de Decisão: 0.8098063076767061
---------- k-nearest neighbors (KNN): 0.7459511115581934
---------- Support vector machine (SVM): 0.7133374183006537

Resumo:

---Naive Bayes:
              precision    recall  f1-score   support

           0       0.74      0.73  